# Create Benchmark Dataset

* need to download below package

In [ ]:
!pip install html2text
!pip install newspaper

---

In [59]:
import requests
import json
import html2text
import random
from newspaper import Article

## function

In [60]:
# crawl by our parser
def get_content(url):
    api = "https://gparsed-admin.gliaoffice.com/parsed/api/parse?url="
    json_file = requests.get(api + url).json()
    return json_file['parsed_content']['content']

In [61]:
# crawl by newspaper
def newspaper(url):
    article = Article(url)
    article.download()
    article.parse()
    return article.text

In [62]:
# filter image text e.g. ![](https://thehill.com/sites/default/files/styles/thumb_100/public/noemkristi_030217gn.jpg?itok=UMIpgu97)
def filter_content(text):
    t = text
    while t.find('![') != -1:
        start = t.find('![')
        step = 1
        stack = 0
        while t[start + step: start + step + 5] != '(http':
            step = step + 1
        stack = stack + 1


        while stack != 0:
            step += 1
            if t[start + step] == '(':
                stack = stack + 1
            elif t[start + step] == ')':
                stack = stack - 1 
        end = start + step
        t = t[:start] + t[end+1:]
    return t

## output json dict


In [63]:
json_dict = {}

之前已經存過檔了，跑下面這個。第一次做的話跳到下一個cell

In [ ]:
your_file_path = ""
with open(ypur_file_path, 'r') as in_file:
    json_dict = json.load(in_file)

In [64]:
template = {"articleBody": "", "url": ""}

## 流程
1. 將url貼上來
    * 連結在`./url.txt`，皆為測試過的有效連結
    * 如果連結還是有問題，可以到https://github.com/livingbio/gparsed/tree/master/src/parsers/test_parsers/test_cases ，每一個資料夾裡面找一個.item檔，立面有網頁連結
    * 有16個資料夾裡連結都失效，有在`url.txt`中註記
2. 執行gparser和newspapers
3. 比對這兩個的原文的結果
    * 選爬到完整或是超過內文的結果再去進行刪減比較快
    * 刪減的時候將print(repr())的結果複製下來到文字編輯器中進行刪減比較快
    * 換行符號、標點符號錯誤不需理會
    * 內文的定義可以查看https://paper.dropbox.com/doc/Benchmark-A.I.-Parser--BWBUq0_rFdVI8Lj3a_rFoq~5Ag-UPbZxMrxYm4XWR452HaTq
    * 範例輸出：https://github.com/livingbio/article-extraction-benchmark/blob/master/ground-truth.json
    * 不確定可以跟我討論
4. 產生key值
5. 將處理好的結果放進template['articleBody'], url放入template['url']
6. 將template放入json_dict[key]
7. back to 1 until you tried

8. 存檔

In [78]:
# url to crawl
url = "https://www.newsobserver.com/news/local/education/article223385845.html"

## 有些網站gparser不能爬，會出現`JSONDecodeError: Expecting value: line 1 column 1 (char 0)`。遇到的時候就直接用newspaper就好。如果gparser和newspaper都failed在跟我說。

In [79]:
gparser = get_content(url)
text = html2text.html2text(gparser)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [57]:
# 可以過濾掉圖片連結
text = filter_content(text)

In [58]:
print(repr(text))

"\n\nSouth Dakota official says meeting with Noem had no effect on daughter's\napplicationSouth Dakota GOP lawmakers summon two employees for Noem\ninquiryRepublicans' mantra should have been 'Stop the Spread'MORE (R) is\ndefending the state’s launch of an anti-drug campaign with the slogan “Meth,\nwe’re on it.”\n\nThe tagline drew a mix of criticism and ridicule across Twitter on Monday, but\nNoem cited the backlash as proof that efforts to raise awareness around South\nDakota's methamphetamine crisis was, in fact, working.\n\n“Meth is IN SD. Twitter can make a joke of it, but when it comes down to it -\nMeth is a serious problem in SD. We are here to Get. It. OUT,” Noem tweeted\nMonday night.\n\nADVERTISEMENT\n\nNoem later tweeted that the state’s meth epidemic “needs to be a dinner table\nconversation,” adding “we need everyone on it.”\n\nThe campaign, which includes both digital and TV ads, cost the state roughly\n$449,000, according to the Sioux Falls Argus Leader.\n\nOne campaign

In [80]:
news = newspaper(url)

ArticleException: Article `download()` failed with HTTPSConnectionPool(host='www.newsobserver.com', port=443): Read timed out. (read timeout=7) on URL https://www.newsobserver.com/news/local/education/article223385845.html

In [68]:
print(repr(news))

'An Indiana toddler had a fatal fall from a Royal Caribbean cruise ship after her grandfather put her up on a railing, thinking the glass window was closed, according to the family\'s attorney.\n\nChloe Wiegand, who was 18 months old, was traveling with her grandparents and parents on the Royal Caribbean Freedom of the Seas when she fell to her death on Sunday, according to her parent\'s attorney, Michael Winkleman.\n\nChloe was with her grandfather in the children\'s water park play area when her grandfather put her on a wood railing in front of a wall of glass windows, Winkleman said at a news conference on Tuesday.\n\nChloe was a hockey fan and loved to bang on the ice hockey rink glass wall as she watched her older brother, Winkleman said, and her grandfather put her on the railing thinking she\'d bang on the glass.\n\nWinkleman told ABC News they are waiting to see closed circuit TV footage of the fall, and have yet to determine whether a lawsuit will be filed.\n\n"The singular re

## generate key

In [69]:
key = str.join('', [ random.choice('abcdefghijklmnopqrstuvwxyz0123456789') for _ in range(64) ])
print(key)

zmgsjj4j8skzkcga5s3x3wz4ez2bkz3vzqri8w14j4y8sw4s5b206sq7lzolzdma


In [73]:
# 放入處理好的文字 或是不需處理的話直接放變數
ok_text = """An Indiana toddler had a fatal fall from a Royal Caribbean cruise ship after her grandfather put her up on a railing, thinking the glass window was closed, according to the family\'s attorney.\n\nChloe Wiegand, who was 18 months old, was traveling with her grandparents and parents on the Royal Caribbean Freedom of the Seas when she fell to her death on Sunday, according to her parent\'s attorney, Michael Winkleman.\n\nChloe was with her grandfather in the children\'s water park play area when her grandfather put her on a wood railing in front of a wall of glass windows, Winkleman said at a news conference on Tuesday.\n\nChloe was a hockey fan and loved to bang on the ice hockey rink glass wall as she watched her older brother, Winkleman said, and her grandfather put her on the railing thinking she\'d bang on the glass.\n\nWinkleman told ABC News they are waiting to see closed circuit TV footage of the fall, and have yet to determine whether a lawsuit will be filed.\n\n"The singular reason they hired me to start was to help with the truth to get out there, because there was a lot of misinformation being spread on the Internet and various sources," Winkleman told ABC News in an interview Tuesday night. "But of course, my primary role as their advocate and the attorney, is to help them seek justice for what happened to Chloe."\n\n"And if that means filing a lawsuit against them, that\'s certainly something that is in the playbook of options," he added.\n\nHowever, the window was open -- and she was "gone," Winkleman said, calling her death a "tragic accident that was preventable."\n\n"Why would you ever in a kids play area put windows that passengers can open?" Winkleman said. "I don\'t think it\'s unreasonable to put a child there thinking there\'s glass."\n\nThe Wiegand family had not opened the window and do not know who opened it, Winkleman said.\n\nThe cruise ship was docked in San Juan, Puerto Rico, at the time, according to the Puerto Rico Department of Public Safety. The Puerto Rico prosecutor has been conducting interviews.\n\nChloe\'s autopsy has been completed, and her remains were sent to a funeral home selected by the family on Tuesday.\n\n“The Department of Justice will continue working on the investigation of this case with the rigor it demands," a spokesperson for the Puerto Rico Department of Justice said in a statement.\n\nThe investigation into the child\'s death is being handled by Puerto Rico Police, and the San Juan attorney general\'s office.\n\nChloe was the daughter of a South Bend, Indiana, police officer, according to the department, and Chloe\'s parents are "beside themselves," Winkleman said.\n\n"The South Bend Police Department offers its sincerest condolences to Officer Alan Wiegand and his family during this difficult time following the tragic loss of their child while in Puerto Rico," the department said in a statement. "The department asks the community to pray for the entire Wiegand family as they grieve and to respect their privacy."\n\nSouth Bend Mayor and presidential candidate Pete Buttigieg added on Twitter, "We are saddened by the terrible accident that took the life of Officer Wiegand\'s young daughter, and the city is holding this family in our hearts."\n\nOwen Torres, corporate communications manager with Royal Caribbean, said in a statement: "We are deeply saddened by yesterday\'s tragic incident, and our hearts go out to the family. We\'ve made our Care Team available to assist the family with any resources they need."\n\nTorres added on Tuesday, "We are assisting local authorities in San Juan, PR, as they make inquiries."""

In [74]:
# 存入json_dict中
template["articleBody"] =  ok_text
template["url"] = url
if key not in json_dict:
    json_dict[key] = template
else:
    print("key exit! Need to generate another")

In [75]:
json_dict

{'zmgsjj4j8skzkcga5s3x3wz4ez2bkz3vzqri8w14j4y8sw4s5b206sq7lzolzdma': {'articleBody': 'An Indiana toddler had a fatal fall from a Royal Caribbean cruise ship after her grandfather put her up on a railing, thinking the glass window was closed, according to the family\'s attorney.\n\nChloe Wiegand, who was 18 months old, was traveling with her grandparents and parents on the Royal Caribbean Freedom of the Seas when she fell to her death on Sunday, according to her parent\'s attorney, Michael Winkleman.\n\nChloe was with her grandfather in the children\'s water park play area when her grandfather put her on a wood railing in front of a wall of glass windows, Winkleman said at a news conference on Tuesday.\n\nChloe was a hockey fan and loved to bang on the ice hockey rink glass wall as she watched her older brother, Winkleman said, and her grandfather put her on the railing thinking she\'d bang on the glass.\n\nWinkleman told ABC News they are waiting to see closed circuit TV footage of the

## Store in json file
* 累了要休工不做了再存進去

In [77]:
your_file_path = "./our_dataset.json"
with open(your_file_path, 'w') as out_file:
    json.dump(json_dict, out_file, ensure_ascii=False, indent=4)